In [2]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

# Load env variables
load_dotenv()
pg_conn_str = f"postgresql+psycopg2://{os.getenv('PG_USER')}:{os.getenv('PG_PASSWORD')}@{os.getenv('PG_HOST')}/{os.getenv('PG_DB')}"
engine = create_engine(pg_conn_str)

# Run raw SQL (e.g., create schema)
with engine.connect() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS staging;"))
    conn.commit()

In [3]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS staging.api_users_cleaned AS
        SELECT 
            id AS user_id,
            TRIM(LOWER(name)) AS name,
            TRIM(LOWER(username)) AS username,
            LOWER(email) AS email,
            phone,
            website,
            address_street,
            address_suite,
            address_city,
            address_zipcode,
            address_geo_lat::FLOAT,
            address_geo_lng::FLOAT,
            company_name,
            company_catchphrase,
            company_bs
        FROM sql_project.api_users;
    """))
    conn.commit()


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "address_street" does not exist
LINE 10:             address_street,
                     ^
HINT:  Perhaps you meant to reference the column "api_users.address.street".

[SQL: 
        CREATE TABLE IF NOT EXISTS staging.api_users_cleaned AS
        SELECT 
            id AS user_id,
            TRIM(LOWER(name)) AS name,
            TRIM(LOWER(username)) AS username,
            LOWER(email) AS email,
            phone,
            website,
            address_street,
            address_suite,
            address_city,
            address_zipcode,
            address_geo_lat::FLOAT,
            address_geo_lng::FLOAT,
            company_name,
            company_catchphrase,
            company_bs
        FROM sql_project.api_users;
    ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [4]:
pd.read_sql("SELECT * FROM sql_project.api_users LIMIT 1", engine)


,id,name,username,email,phone,website,address.street,address.suite,address.city,address.zipcode,address.geo.lat,address.geo.lng,company.name,company.catchPhrase,company.bs
0,1,Leanne Graham,Bret,Sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets


In [5]:
from sqlalchemy import text

with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS staging.api_users_cleaned AS
        SELECT 
            id AS user_id,
            TRIM(LOWER(name)) AS name,
            TRIM(LOWER(username)) AS username,
            LOWER(email) AS email,
            phone,
            website,
            "address.street" AS address_street,
            "address.suite" AS address_suite,
            "address.city" AS address_city,
            "address.zipcode" AS address_zipcode,
            "address.geo.lat" AS address_geo_lat,
            "address.geo.lng" AS address_geo_lng,
            "company.name" AS company_name,
            "company.catchPhrase" AS company_catchphrase,
            "company.bs" AS company_bs
        FROM sql_project.api_users;
    """))
    conn.commit()


In [7]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS staging.api_comments_cleaned AS
        SELECT 
            "postId"::INT AS post_id,
            id::INT AS comment_id,
            TRIM(name) AS commenter_name,
            LOWER(email) AS commenter_email,
            TRIM(body) AS comment_text
        FROM sql_project.api_comments;
    """))
    conn.commit()


In [8]:
pd.read_sql("SELECT * FROM staging.api_comments_cleaned LIMIT 5", engine)


,post_id,comment_id,commenter_name,commenter_email,comment_text
0,1,1,id labore ex et quam laborum,eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...
1,1,2,quo vero reiciendis velit similique earum,jayne_kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...
2,1,3,odio adipisci rerum aut animi,nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...
3,1,4,alias odio sit,lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...
4,1,5,vero eaque aliquid doloribus et culpa,hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...


In [9]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS staging.api_users_cleaned AS
        SELECT 
            id::INT AS user_id,
            TRIM(name) AS full_name,
            LOWER(email) AS user_email,
            TRIM(username) AS username,
            TRIM(phone) AS phone,
            TRIM(website) AS website,
            TRIM("address.street") AS address_street,
            TRIM("address.city") AS address_city,
            TRIM("address.zipcode") AS address_zipcode,
            "address.geo.lat"::FLOAT AS geo_lat,
            "address.geo.lng"::FLOAT AS geo_lng,
            TRIM("company.name") AS company_name,
            TRIM("company.catchPhrase") AS company_catchphrase,
            TRIM("company.bs") AS company_bs
        FROM sql_project.api_users;
    """))
    conn.commit()


In [10]:
pd.read_sql("SELECT * FROM staging.api_users_cleaned LIMIT 5", engine)


,user_id,name,username,email,phone,website,address_street,address_suite,address_city,address_zipcode,address_geo_lat,address_geo_lng,company_name,company_catchphrase,company_bs
0,1,leanne graham,bret,sincere@april.biz,1-770-736-8031 x56442,hildegard.org,Kulas Light,Apt. 556,Gwenborough,92998-3874,-37.3159,81.1496,Romaguera-Crona,Multi-layered client-server neural-net,harness real-time e-markets
1,2,ervin howell,antonette,shanna@melissa.tv,010-692-6593 x09125,anastasia.net,Victor Plains,Suite 879,Wisokyburgh,90566-7771,-43.9509,-34.4618,Deckow-Crist,Proactive didactic contingency,synergize scalable supply-chains
2,3,clementine bauch,samantha,nathan@yesenia.net,1-463-123-4447,ramiro.info,Douglas Extension,Suite 847,McKenziehaven,59590-4157,-68.6102,-47.0653,Romaguera-Jacobson,Face to face bifurcated interface,e-enable strategic applications
3,4,patricia lebsack,karianne,julianne.oconner@kory.org,493-170-9623 x156,kale.biz,Hoeger Mall,Apt. 692,South Elvis,53919-4257,29.4572,-164.2990,Robel-Corkery,Multi-tiered zero tolerance productivity,transition cutting-edge web services
4,5,chelsey dietrich,kamren,lucio_hettinger@annie.ca,(254)954-1289,demarco.info,Skiles Walks,Suite 351,Roscoeview,33263,-31.8129,62.5342,Keebler LLC,User-centric fault-tolerant solution,revolutionize end-to-end systems


In [15]:
with engine.connect() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS warehouse;"))
    conn.commit()


In [16]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS warehouse.dim_users AS
        SELECT 
            user_id,
            name,
            username,
            email,
            phone,
            website,
            address_street,
            address_suite,
            address_city,
            address_zipcode,
            address_geo_lat,
            address_geo_lng,
            company_name,
            company_catchphrase,
            company_bs
        FROM staging.api_users_cleaned;
    """))
    conn.commit()


In [17]:
with engine.connect() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS warehouse.fact_comments AS
        SELECT 
            comment_id,
            commenter_name,
            commenter_email,
            comment_text,
            post_id AS user_id
        FROM staging.api_comments_cleaned;
    """))
    conn.commit()


In [18]:
pd.read_sql("SELECT * FROM warehouse.fact_comments LIMIT 5", engine)


,comment_id,commenter_name,commenter_email,comment_text,user_id
0,1,id labore ex et quam laborum,eliseo@gardner.biz,laudantium enim quasi est quidem magnam volupt...,1
1,2,quo vero reiciendis velit similique earum,jayne_kuhic@sydney.com,est natus enim nihil est dolore omnis voluptat...,1
2,3,odio adipisci rerum aut animi,nikita@garfield.biz,quia molestiae reprehenderit quasi aspernatur\...,1
3,4,alias odio sit,lew@alysha.tv,non et atque\noccaecati deserunt quas accusant...,1
4,5,vero eaque aliquid doloribus et culpa,hayden@althea.biz,harum non quasi et ratione\ntempore iure ex vo...,1


##Business Question: 
Which users receive the most comments and what can we learn about their visibility or engagement?

In [ ]:
Query = '''WITH comment_counts AS (
    SELECT 
        u.user_id,
        u.name,
        u.email,
        COUNT(c.comment_id) AS total_comments
    FROM warehouse.dim_users u
    LEFT JOIN warehouse.fact_comments c 
        ON u.user_id = c.user_id
    GROUP BY u.user_id, u.name, u.email
),
ranked_users AS (
    SELECT *,
        RANK() OVER (ORDER BY total_comments DESC) AS rank_by_comments
    FROM comment_counts
)
SELECT *
FROM ranked_users
ORDER BY rank_by_comments; '''

In [1]:
import pandas as pd
pd.set_option('display.max_rows', None)


Insights:
By joining user records with comments, we identified the top 10 users who have received the highest number of comments. Surprisingly, all top users have received exactly 5 comments, which suggests a uniform distribution of engagement across the dataset.

Recommendation:
If using mock data for modeling, simulate a more realistic distribution of comments to better reflect varied user experiences.

For future data sourcing, expand beyond uniform APIs like JSONPlaceholder to introduce variation that enables real-world analysis (e.g., identify high-touch vs. low-touch customers).

Prediction: Based on the uniform distribution of comments across users, I predict that if real engagement data were used, we would observe a long-tail pattern — where a small number of users generate a majority of comments. This could help prioritize support resources for high-engagement customers.
